In [2]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [107]:
train_path = r'B:\images\output\train'
test_path = r'B:\images\output\test'
val_path = r'B:\images\output\val'
image_size = (256, 256)
batch_size = 32

In [11]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(images_path, seed=123, image_size=image_size, batch_size=batch_size, label_mode='categorical')

Found 282494 files belonging to 20 classes.
Using 225996 files for training.
Using 56498 files for validation.


In [104]:
import splitfolders

In [138]:
train_ds = tf.keras.utils.image_dataset_from_directory(train_path, seed=123, image_size=image_size, batch_size=batch_size, label_mode='categorical')

Found 225986 files belonging to 20 classes.


In [136]:
test_ds = tf.keras.utils.image_dataset_from_directory(test_path, seed=123, image_size=image_size, batch_size=batch_size, label_mode='categorical', shuffle=False)

Found 28266 files belonging to 20 classes.


In [137]:
val_ds = tf.keras.utils.image_dataset_from_directory(val_path, seed=123, image_size=image_size, batch_size=batch_size, label_mode='categorical', shuffle=False)

Found 28242 files belonging to 20 classes.


In [114]:
print(train_ds.class_names, test_ds.class_names, val_ds.class_names)

['2k22', 'AmongUs', 'Apex', 'Ark', 'B4B', 'CSGO', 'Deducto', 'DeepRock', 'GTAV', 'GreenHell', 'LoL', 'Minecraft', 'Overwatch', 'Satisfactory', 'SeaOfThieves', 'Tribes', 'Valheim', 'Valorant', 'Warzone', 'WoW'] ['2k22', 'AmongUs', 'Apex', 'Ark', 'B4B', 'CSGO', 'Deducto', 'DeepRock', 'GTAV', 'GreenHell', 'LoL', 'Minecraft', 'Overwatch', 'Satisfactory', 'SeaOfThieves', 'Tribes', 'Valheim', 'Valorant', 'Warzone', 'WoW'] ['2k22', 'AmongUs', 'Apex', 'Ark', 'B4B', 'CSGO', 'Deducto', 'DeepRock', 'GTAV', 'GreenHell', 'LoL', 'Minecraft', 'Overwatch', 'Satisfactory', 'SeaOfThieves', 'Tribes', 'Valheim', 'Valorant', 'Warzone', 'WoW']


In [139]:
train_ds = train_ds.map(lambda x, y: (tf.divide(x, 255), y))
test_ds = test_ds.map(lambda x, y: (tf.divide(x, 255), y))
val_ds = val_ds.map(lambda x, y: (tf.divide(x, 255), y))

In [140]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [151]:
x = keras.Sequential()

In [152]:
x.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
x.add(keras.layers.MaxPooling2D())
x.add(keras.layers.Dropout(0.25))
x.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
x.add(keras.layers.MaxPooling2D())
x.add(keras.layers.Dropout(0.5))
x.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
x.add(keras.layers.MaxPooling2D())
x.add(keras.layers.Dropout(0.5))
x.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
x.add(keras.layers.MaxPooling2D())
x.add(keras.layers.Dropout(0.5))
x.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
x.add(keras.layers.Flatten())
x.add(keras.layers.Dropout(0.5))
x.add(keras.layers.Dense(20, activation='softmax'))

In [153]:
epochs = 15

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"), keras.callbacks.EarlyStopping()
]

In [154]:
x.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [155]:
with tf.device('/gpu:0'):
    x.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds, batch_size = batch_size
    )

Epoch 1/15
 441/7063 [>.............................] - ETA: 34:45 - loss: 0.4990 - accuracy: 0.8469

KeyboardInterrupt: 

In [27]:
results = x.evaluate()

{'loss': [0.030997568741440773, 0.011825117282569408],
 'accuracy': [0.9924600720405579, 0.9970707297325134],
 'val_loss': [0.017801469191908836, 0.028335614129900932],
 'val_accuracy': [0.9962653517723083, 0.9954334497451782]}

In [30]:
import cv2

In [90]:
x_test = r'B:\images\test'

In [46]:
y_test = np.array(['CSGO']*1440, dtype=list)

In [47]:
y_test

array(['CSGO', 'CSGO', 'CSGO', ..., 'CSGO', 'CSGO', 'CSGO'], dtype=object)

In [51]:
results = x.predict(val_ds)

1766/1766 [==============================] - 100s 56ms/step


In [59]:
classs = np.argmax(results, axis=1)

In [62]:
classs

array([17,  5, 12, ...,  5, 19, 17], dtype=int64)

In [82]:
for images, labels in val_ds.take(1):  # only take first element of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

In [96]:
test_ds = tf.keras.utils.image_dataset_from_directory(x_test, label_mode='categorical', image_size=image_size)

Found 167 files belonging to 1 classes.


In [101]:
test_results = x.evaluate(test_ds)

6/6 [==============================] - 0s 23ms/step - loss: 689.6262 - accuracy: 0.0000e+00


In [102]:
test_results

[689.626220703125, 0.0]